In [ ]:
# 获取当前notebook的绝对路径
import sys, os
# import asyncio
# import nest_asyncio
# nest_asyncio.apply()
from pathlib import Path
current_path = Path.cwd()  # 或者使用 %pwd magic命令
project_root = current_path.parents[4]  # 如果notebook在project_root/notebooks/下
print("notebook所在的路径：", current_path)
print("project_root：", project_root)
sys.path.append(str(project_root))
import FastAPI_setup

In [ ]:
# agent get document 在 agent已经测试通过，这里拿过来使用
# project_id = "20563b61-7818-474b-9641-c40757bbd3fe"  # test1
project_id = "cc6bca8a-df5d-43d5-a931-875113df1f50"  # test2
from app.services.structuring.agent import DocumentStructureAgent
agent = DocumentStructureAgent(project_id)
document = await agent.state_manager.cache.get_document('raw_document')
from pprint import pprint
pprint(f"document: {document}")

In [3]:
from app.services.structuring.prompts.tender_outlines_L1 import TenderOutlinesL1PromptBuilder
prompt_builder = TenderOutlinesL1PromptBuilder(document)

In [ ]:
prompt_config, task_inputs, meta = await prompt_builder.output_params()
pprint(f"output_params:{prompt_config}")
pprint(f"output_params: {task_inputs}")
pprint(f"output_params: {meta}")


In [ ]:
pprint(task_inputs)